In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from functools import partial

# Load dataset
dataset = load_dataset("OpenAssistant/oasst1", split="train")

# Load tokenizer and model
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Get the PEFT model
model = get_peft_model(model, lora_config)

# Tokenize function
def tokenize_function(examples, max_length=512):  # Reduced from 2048
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)

# Tokenize the dataset
tokenized_dataset = dataset.map(
    partial(tokenize_function, max_length=512),
    batched=True,
    remove_columns=dataset.column_names,
    num_proc=4  # Use multiple processes for faster preprocessing
)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # Reduced from 3
    per_device_train_batch_size=8,  # Increased from 4
    gradient_accumulation_steps=2,  # Reduced from 4
    warmup_steps=100,  # Reduced from 500
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=5e-5,  # Increased from 2e-5
    fp16=True,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    weight_decay=0.01,
    remove_unused_columns=False,  # Added to address a potential warning
    optim="adamw_torch",  # Explicitly set optimizer
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./phi_1_5_finetuned")
tokenizer.save_pretrained("./phi_1_5_finetuned")

print("Fine-tuning complete. Model saved to ./phi_1_5_finetuned")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Map (num_proc=4):   0%|          | 0/84437 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnin

Step,Training Loss
10,2.354800
20,2.502200
30,2.388400
40,2.724400
50,2.479500
60,2.469200
70,2.290800
80,2.247600
90,2.231500
100,2.163000


Fine-tuning complete. Model saved to ./phi_1_5_finetuned


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from huggingface_hub import HfApi, login
import torch

def get_huggingface_token():
    print("Please enter your Hugging Face token.")
    print("You can find your token at https://huggingface.co/settings/tokens")
    return input("Token: ").strip()

def get_huggingface_username():
    return input("Please enter your Hugging Face username: ").strip()

# Try to login, if it fails, ask for token
try:
    login()
except:
    token = get_huggingface_token()
    login(token=token)

# Get the username
username = get_huggingface_username()

# Load your fine-tuned model and tokenizer
local_model_path = "./phi_1_5_finetuned"  # Path to your locally saved model
config = PeftConfig.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, local_model_path)
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Set up the model card (same as before)
model_card = """
# ... (your model card content here)
"""

# Set up repository name with the provided username
repo_name = f"{username}/phi-1_5-finetuned"

# Push the model and tokenizer to the Hub
model.push_to_hub(repo_name, use_auth_token=True)
tokenizer.push_to_hub(repo_name, use_auth_token=True)

# Create and push the model card
api = HfApi()
api.create_repo(repo_name, exist_ok=True)
api.upload_file(
    path_or_fileobj=model_card.encode(),
    path_in_repo="README.md",
    repo_id=repo_name,
    commit_message="Add model card"
)

print(f"Model, tokenizer, and model card pushed to https://huggingface.co/{repo_name}")

Please enter your Hugging Face username:  sagar007


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/hf_api.py:3757: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


Model, tokenizer, and model card pushed to https://huggingface.co/sagar007/phi-1_5-finetuned


In [11]:
#!pip install transformers peft -q

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from google.colab import output
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the fine-tuned model and tokenizer
model_name = "phi_1_5_finetuned"  # Replace with your actual model path on Hugging Face

print("Loading model and tokenizer...")
config = PeftConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    torch_dtype=torch.float16, 
    device_map="auto"
)
model = PeftModel.from_pretrained(model, model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Model and tokenizer loaded successfully.")

def generate_text(prompt, max_length=100, temperature=0.7, top_p=0.9):
    """Generate text based on the input prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Create input widgets
prompt_input = widgets.Textarea(description="Prompt:", layout=widgets.Layout(width='500px'))
max_length_input = widgets.IntSlider(value=100, min=10, max=500, step=10, description="Max Length:")
temperature_input = widgets.FloatSlider(value=0.7, min=0.1, max=1.0, step=0.1, description="Temperature:")
top_p_input = widgets.FloatSlider(value=0.9, min=0.1, max=1.0, step=0.1, description="Top P:")
generate_button = widgets.Button(description="Generate Text")
output_area = widgets.Textarea(layout=widgets.Layout(width='500px', height='200px'))

# Display widgets
display(prompt_input, max_length_input, temperature_input, top_p_input, generate_button, output_area)

# Define button click event
def on_button_click(b):
    prompt = prompt_input.value
    max_length = max_length_input.value
    temperature = temperature_input.value
    top_p = top_p_input.value
    
    output_area.value = "Generating text..."
    
    generated_text = generate_text(prompt, max_length, temperature, top_p)
    
    output_area.value = generated_text

generate_button.on_click(on_button_click)

Loading model and tokenizer...
Model and tokenizer loaded successfully.


Textarea(value='', description='Prompt:', layout=Layout(width='500px'))

IntSlider(value=100, description='Max Length:', max=500, min=10, step=10)

FloatSlider(value=0.7, description='Temperature:', max=1.0, min=0.1)

FloatSlider(value=0.9, description='Top P:', max=1.0, min=0.1)

Button(description='Generate Text', style=ButtonStyle())

Textarea(value='', layout=Layout(height='200px', width='500px'))

In [9]:
# Save the model
output_dir = "./phi-1.5-fine-tuned_kaggle"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to ./phi-1.5-fine-tuned_kaggle
